##Read Bronze table

In [0]:

import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import col, trim, length


df = spark.table("workspace.bronze.erp_loc_a101")

In [0]:
df.display()

CID,CNTRY
AW-00011000,Australia
AW-00011001,Australia
AW-00011002,Australia
AW-00011003,Australia
AW-00011004,Australia
AW-00011005,Australia
AW-00011006,Australia
AW-00011007,Australia
AW-00011008,Australia
AW-00011009,Australia


##Silver Transformations

In [0]:
##trimimg

for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

In [0]:
##date cleaning

df = df.withColumn("cid", F.regexp_replace(col("cid"), "-", ""))

In [0]:
##country normalization

df = df.withColumn(
    "cntry",
    F.when(col("cntry") == "DE", "Germany")
     .when(col("cntry").isin("US", "USA"), "United States")
     .when((col("cntry") == "") | col("cntry").isNull(), "n/a")
     .otherwise(col("cntry"))
)

In [0]:
##rename columns
RENAME_MAP = {
    "cid": "customer_number",
    "cntry": "country"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

In [0]:
df.display ()

customer_number,country
AW00011000,Australia
AW00011001,Australia
AW00011002,Australia
AW00011003,Australia
AW00011004,Australia
AW00011005,Australia
AW00011006,Australia
AW00011007,Australia
AW00011008,Australia
AW00011009,Australia


In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_loc")